In [1]:
# Section B

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import t
from math import sqrt


In [118]:
# Task 1
# The following parts are calculating theoretical values for VaR and CVaR 
# for various normal and student-t distributions.

def theory_value_normal(mu,sigma,q_bar):
    '''
    input: mu, sigma for normal distribution; q_bar, for quantile
    output: VaR and CVaR (4 digits after the decimal)
    '''
    m, s = mu, sigma
    alpha=1-q_bar

    VaR=norm.ppf(1-alpha)*s+m
    CVaR=alpha**(-1)*norm.pdf(norm.ppf(alpha))*s+m
    
    print(' Under q_bar = {}, VaR for N({},{}) is {:.4f}'.format(q_bar,m,s,VaR))
    print(' Under q_bar = {}, CVaR for N({},{}) is {:.4f}'.format(q_bar,m,s,CVaR))

theory_value_normal(0,1,0.95)
theory_value_normal(1,2,0.95)
theory_value_normal(3,5,0.95)
theory_value_normal(0,1,0.99)
theory_value_normal(1,2,0.99)
theory_value_normal(3,5,0.99)


 Under q_bar = 0.95, VaR for N(0,1) is 1.6449
 Under q_bar = 0.95, CVaR for N(0,1) is 2.0627
 Under q_bar = 0.95, VaR for N(1,2) is 4.2897
 Under q_bar = 0.95, CVaR for N(1,2) is 5.1254
 Under q_bar = 0.95, VaR for N(3,5) is 11.2243
 Under q_bar = 0.95, CVaR for N(3,5) is 13.3136
 Under q_bar = 0.99, VaR for N(0,1) is 2.3263
 Under q_bar = 0.99, CVaR for N(0,1) is 2.6652
 Under q_bar = 0.99, VaR for N(1,2) is 5.6527
 Under q_bar = 0.99, CVaR for N(1,2) is 6.3304
 Under q_bar = 0.99, VaR for N(3,5) is 14.6317
 Under q_bar = 0.99, CVaR for N(3,5) is 16.3261


In [119]:
def theory_value_t(df,q_bar):
    '''
    input: df, degrees of freedom for t-distribution; q_bar, for quantile
    output: VaR and CVaR (4 digits after the decimal)
    '''
    m=0
    s=sqrt(df/(df-2))
    alpha=1-q_bar
    degf=df # degrees of freedom
    #reverse_cdf=t.ppf(alpha, degf)
    reverse_cdf=t.ppf(q_bar, degf)
    x = np.random.standard_t(degf,100000)

    VaR=sqrt((degf-2)/degf)*t.ppf(1-alpha, degf)*s-m
    CVaR = np.mean(np.maximum(-VaR-x,0)) / (1-q_bar) + VaR
    print(' Under q_bar = {}, VaR for d.f.={} is {:.4f}'.format(q_bar,df,VaR))
    print(' Under q_bar = {}, CVaR for d.f.={} is {:.4f}'.format(q_bar,df,CVaR))

theory_value_t(10,0.95)
theory_value_t(7,0.95)
theory_value_t(3,0.95)
theory_value_t(10,0.99)
theory_value_t(7,0.99)
theory_value_t(3,0.99)

 Under q_bar = 0.95, VaR for d.f.=10 is 1.8125
 Under q_bar = 0.95, CVaR for d.f.=10 is 2.4212
 Under q_bar = 0.95, VaR for d.f.=7 is 1.8946
 Under q_bar = 0.95, CVaR for d.f.=7 is 2.5972
 Under q_bar = 0.95, VaR for d.f.=3 is 2.3534
 Under q_bar = 0.95, CVaR for d.f.=3 is 3.8582
 Under q_bar = 0.99, VaR for d.f.=10 is 2.7638
 Under q_bar = 0.99, CVaR for d.f.=10 is 3.3180
 Under q_bar = 0.99, VaR for d.f.=7 is 2.9980
 Under q_bar = 0.99, CVaR for d.f.=7 is 3.7507
 Under q_bar = 0.99, VaR for d.f.=3 is 4.5407
 Under q_bar = 0.99, CVaR for d.f.=3 is 7.1296


In [158]:
# The following parts are calculating VaR and CVaR for various normal 
# and student-t distributions by using SGLD method.

# Notice: Based on the author stated in the paper, the original times of 
# iteration is n=1e6 and need 10000 samples. But my computer's performance 
# is weak, it will take a huge amount of time, for more than 3 hours, to just 
# calculate a single result if I run for such n*10000 loops. So I have to 
# reduce the loop, as the result, the value I calculated may not be the same 
# shown in the paper. Hope you would not mind. :-)

# I change n=1e4 for efficiency
n=10000 
# The same reason above, for efficiency I take 10 samples.
loop = 10  # 10 rounds for 1 minute
res = np.zeros((loop))
theta=np.zeros((n))
beta=1e8
gamma=1e-8
lamda=1e-4
q_bar=0.95

def indicator(fx,thetaa):
    if fx < thetaa:
        return(1)
    else:
        return(0)

def H(thetaaa,x):
    return(-q_bar/(1-q_bar)+indicator(x,thetaaa)/(1-q_bar)+2*gamma*thetaaa)

def V(heta,mu,sigma):
    x = np.random.normal(mu,sigma,100000)
    return( np.mean(heta + 1/(1-q_bar) * np.maximum(x-heta,0)) + gamma*abs(heta)**2 )

def VaR_SGLD(m,s,q_bar):

    for l in range(loop):
        V_value = np.zeros((n))
        V_value[0] = V(0,m,s)
        for i in range(n-1):
            theta[i+1]=theta[i]-lamda*H(theta[i],np.random.normal(m,s))+sqrt(2*lamda/beta)*np.random.normal(0,1)
            V_value[i+1] = V(theta[i+1],m,s)

        min_V_index = np.argmin(V_value)
        res[l] = theta[min_V_index]


    VaR=sum(res)/loop
    print(' Under q_bar = {}, VaR_SGLD for N({},{}) is {:.4f}, with variance {}'.format(q_bar,m,s,VaR,np.var(res)))

def CVaR_SGLD(m,s,q_bar):

    for l in range(loop):
        V_value = np.zeros((n))
        V_value[0] = V(0,m,s)

        for i in range(n-1):
            theta[i+1] = theta[i]-lamda*H(theta[i],np.random.normal(0,1))+sqrt(2*lamda/beta)*np.random.normal(0,1)
            V_value[i+1] = V(theta[i+1],m,s)
        
        res[l]=min(V_value)

    CVaR=sum(res)/loop
    print(' Under q_bar = {}, CVaR_SGLD for N({},{}) is {:.4f}, with variance {}'.format(q_bar,m,s,CVaR,np.var(res)))

VaR_SGLD(0,1,0.95)
#VaR_SGLD(1,2,0.95)
#VaR_SGLD(3,5,0.95)
#VaR_SGLD(0,1,0.99)
#VaR_SGLD(1,2,0.99)
VaR_SGLD(3,5,0.99)
CVaR_SGLD(0,1,0.95)
#CVaR_SGLD(1,2,0.95)
#CVaR_SGLD(3,5,0.95)
#CVaR_SGLD(0,1,0.99)
#CVaR_SGLD(1,2,0.99)
CVaR_SGLD(3,5,0.99)

 Under q_bar = 0.95, VaR_SGLD for N(0,1) is 1.5580, with variance 0.0002806441544225068
 Under q_bar = 0.99, VaR_SGLD for N(3,5) is 6.9881, with variance 0.0025204118199253184
 Under q_bar = 0.95, CVaR_SGLD for N(0,1) is 2.0485, with variance 3.74643746668882e-05
 Under q_bar = 0.99, CVaR_SGLD for N(3,5) is 56.8773, with variance 0.04166496275929441


In [130]:
n=10000 # 1e6 rounds for 6 seconds
loop = 10  # 10 rounds for 1 minute
res = np.zeros((loop))
theta=np.zeros((n))
beta=1e8
gamma=1e-8
lamda=1e-4
q_bar=0.95
mu,sigma = 0,1

def V(heta,degf):
    #return(heta+1/(1-q_bar)*(mu-mu*norm.cdf(heta,mu,sigma)-heta+heta*norm.cdf(heta,mu,sigma))+gamma*abs(heta)**2)
    #x = np.random.normal(mu,sigma,100000)
    x = np.random.standard_t(degf,100000)
    #return(heta+1/(1-q_bar)*(mu-mu*norm.cdf(heta,mu,sigma)-heta+heta*norm.cdf(heta,mu,sigma))+gamma*abs(heta)**2)
    #return(heta+1/(1-q_bar)*(mu-heta)*(1-norm.cdf(heta,mu,sigma))+gamma*abs(heta)**2)
    return( np.mean(heta + 1/(1-q_bar) * np.maximum(x-heta,0)) + gamma*abs(heta)**2 )

def indicator(fx,thetaa):
    if fx < thetaa:
        return(1)
    else:
        return(0)

def H(thetaaa,x):
    return(-q_bar/(1-q_bar)+indicator(x,thetaaa)/(1-q_bar)+2*gamma*thetaaa)


def VaR_SGLD(degf,q_bar):

    for l in range(loop):
        V_value = np.zeros((n))
        V_value[0] = V(0,degf)
        for i in range(n-1):
            theta[i+1]=theta[i]-lamda*H(theta[i],np.random.standard_t(degf,1))+sqrt(2*lamda/beta)*np.random.normal(0,1)
            V_value[i+1] = V(theta[i+1],degf)

        min_V_index = np.argmin(V_value)
        res[l] = theta[min_V_index]


    VaR=sum(res)/loop
    print(' Under q_bar = {}, VaR_SGLD for t(d.f.={}) is {:.4f}, with variance {}'.format(q_bar,degf,VaR,np.var(res)))

def CVaR_SGLD(degf,q_bar):

    for l in range(loop):
        V_value = np.zeros((n))
        V_value[0] = V(0,degf)

        for i in range(n-1):
            theta[i+1] = theta[i]-lamda*H(theta[i],np.random.standard_t(degf,1))+sqrt(2*lamda/beta)*np.random.normal(0,1)
            V_value[i+1] = V(theta[i+1],degf)
        
        res[l]=min(V_value)

    CVaR=sum(res)/loop
    print(' Under q_bar = {}, CVaR_SGLD for t(d.f.={}) is {:.4f}, with variance {}'.format(q_bar,degf,CVaR,np.var(res)))


VaR_SGLD(10,0.95)
#VaR_SGLD(7,0.95)
#VaR_SGLD(3,0.95)
#VaR_SGLD(10,0.99)
#VaR_SGLD(7,0.99)
VaR_SGLD(3,0.99)
CVaR_SGLD(10,0.95)
#CVaR_SGLD(7,0.95)
#CVaR_SGLD(3,0.95)
#CVaR_SGLD(10,0.99)
#CVaR_SGLD(7,0.99)
CVaR_SGLD(3,0.99)


 Under q_bar = 0.95, VaR_SGLD for t(d.f.=10) is 1.6431, with variance 0.0008927075204260096


In [161]:
# Task 2
# I take n=1000.



def omega_generate(N):
    '''
    Input: N, the number of portfolio composition
    Output: the weights for omega1~3 for three assets 
    '''
    p1 = np.random.uniform(0,1,N)
    p2 = np.random.uniform(0,1,N)
    point = np.sort([p1,p2],axis=0)

    allomega1 = point[0,:]
    allomega2 = point[1,:] - point[0,:]
    allomega3 = 1 - point[1,:]

    return(allomega1,allomega2,allomega3)

def sum_of_gomegax(omega1,omega2,omega3,x1,x2,x3):
    '''
    Input: omega_i, omega for each asset
           x_i, generated sample for each asset distribution
    Output: sum of g(omega)*X_i
    '''
    g1 = np.exp(omega1) / sum(np.exp([omega1,omega2,omega3]))
    g2 = np.exp(omega2) / sum(np.exp([omega1,omega2,omega3]))
    g3 = np.exp(omega3) / sum(np.exp([omega1,omega2,omega3]))

    return(g1*x1+g2*x2+g3*x3)

def Indicator(sumofgx,thetaa):
    '''
    Input: sumofgx, sum of g(omega)*X_i
           thetaa, the value of theta
    Output: 0 or 1
    '''
    if sumofgx >= thetaa:
        return(1)
    else:
        return(0)

def H_d_theta(thetaaa,sumofgx):
    '''
    Input: thetaaa, the value of theta
           sumofgx, sum of g(omega)*X_i
    Output: H derivative of theta
    '''
    return(1 - 1/(1-q_bar) * Indicator(sumofgx,thetaaa) + 2*gamma*thetaaa)

def g(i,omega1,omega2,omega3):
    '''
    Input: i, to decide output which g_i(omega) i=1,2,3
           omega_i, omega for each asset
    Output: g_i(omega)
    '''
    if i == 1:
        return(np.exp(omega1) / (np.exp(omega1)+np.exp(omega2)+np.exp(omega3)))
    if i == 2:
        return(np.exp(omega2) / (np.exp(omega1)+np.exp(omega2)+np.exp(omega3)))
    if i == 3:
        return(np.exp(omega3) / (np.exp(omega1)+np.exp(omega2)+np.exp(omega3)))

    
def V_hat(theta_hat_column,omega1,omega2,omega3):
#def V_hat(theta_hat_column):
    '''
    Input: theta_hat_column, one vector contains [theta,omega1,omega2,omega3]
    Output: V(theta_hat) value
    '''
    new_m = g(1,omega1,omega2,omega3)*m1+g(1,omega1,omega2,omega3)*m2+g(1,omega1,omega2,omega3)*m3
    new_s = np.sqrt(s1*g(1,omega1,omega2,omega3)**2+s2*g(1,omega1,omega2,omega3)**2+s3*g(1,omega1,omega2,omega3)**2)
    new_x = np.random.normal(new_m,new_s,10000)   
    return( np.mean( 1/(1-q_bar) * np.maximum( new_x-theta_hat_column[0],0 ) + theta_hat_column[0] ) + gamma*np.linalg.norm(theta_hat_column,ord=2)**2 )


def H_d_omega(omega1,omega2,omega3,sumofgx,thetaaa,x1,x2,x3):
    g1d1 =  (np.exp(omega1) * (np.exp(omega2)+np.exp(omega3)) / ((np.exp(omega1)+np.exp(omega2)+np.exp(omega3))**2))
    g2d1 = -(np.exp(omega1) * np.exp(omega2) / ((np.exp(omega1)+np.exp(omega2)+np.exp(omega3))**2))
    g3d1 = -(np.exp(omega1) * np.exp(omega3) / ((np.exp(omega1)+np.exp(omega2)+np.exp(omega3))**2))
    
    g1d2 = -(np.exp(omega2) * np.exp(omega1) / ((np.exp(omega1)+np.exp(omega2)+np.exp(omega3))**2))
    g2d2 =  (np.exp(omega2) * (np.exp(omega1)+np.exp(omega3)) / ((np.exp(omega1)+np.exp(omega2)+np.exp(omega3))**2))
    g3d2 = -(np.exp(omega2) * np.exp(omega3) / ((np.exp(omega1)+np.exp(omega2)+np.exp(omega3))**2))
    
    g1d3 = -(np.exp(omega3) * np.exp(omega1) / ((np.exp(omega1)+np.exp(omega2)+np.exp(omega3))**2))
    g2d3 = -(np.exp(omega3) * np.exp(omega2) / ((np.exp(omega1)+np.exp(omega2)+np.exp(omega3))**2))
    g3d3 =  (np.exp(omega3) * (np.exp(omega1)+np.exp(omega2)) / ((np.exp(omega1)+np.exp(omega2)+np.exp(omega3))**2))

    gd1 = g1d1*x1+g2d1*x2+g3d1*x3
    gd2 = g1d2*x1+g2d2*x2+g3d2*x3
    gd3 = g1d3*x1+g2d3*x2+g3d3*x3

    Hd1 = 1/(1-q_bar) * gd1 *Indicator(sumofgx,thetaaa) + 2*gamma*omega1
    Hd2 = 1/(1-q_bar) * gd2 *Indicator(sumofgx,thetaaa) + 2*gamma*omega2
    Hd3 = 1/(1-q_bar) * gd3 *Indicator(sumofgx,thetaaa) + 2*gamma*omega3

    return(Hd1,Hd2,Hd3)


#n should be 1e6
n=1000
N=100
#N=100 # 100 kinds of portfolio composition
# theta_hat: [theta,omega1,omega2,omega3]
theta_hat = np.zeros((4,n))
beta=1e8
gamma=1e-8
lamda=1e-4
q_bar=0.95
# manually change coefficients

#m1,m2,m3,s1,s2,s3 = 500,0,0,1,1e6,1e-4
#m1,m2,m3,s1,s2,s3 = 500,0,0,1,1e6,1
m1,m2,m3,s1,s2,s3 = 0,0,0,1e3,1,4
#m1,m2,m3,s1,s2,s3 = 0,1,0,1,4,1e-4
#m1,m2,m3,s1,s2,s3 = 0,1,2,1,4,1
s1_std,s2_std,s3_std = np.sqrt(s1),np.sqrt(s2),np.sqrt(s3)
res = np.zeros((N))
mintheta = np.zeros((N))

for s in range(N):
    allomega1,allomega2,allomega3 = omega_generate(N)
    omega1,omega2,omega3 = allomega1[s],allomega2[s],allomega3[s]
    theta_hat[1:,0] = [omega1,omega2,omega3] 
    V_hat_value = np.zeros((n))
    V_hat_value[0] = V_hat(theta_hat[:,0],omega1,omega2,omega3)
    #V_hat_value[0] = V_hat(theta_hat[:,0])
    for i in range(n-1):
        x1 = np.random.normal(m1,s1_std)
        x2 = np.random.normal(m2,s2_std)
        x3 = np.random.normal(m3,s3_std)

        sumofgx = sum_of_gomegax(omega1,omega2,omega3,x1,x2,x3)
        Hdtheta = H_d_theta(theta_hat[0,i],sumofgx)
        Hd1,Hd2,Hd3 = H_d_omega(omega1,omega2,omega3,sumofgx,theta_hat[0,i],x1,x2,x3)

        theta_hat[:,i+1] = theta_hat[:,i] - lamda*np.array([Hdtheta,Hd1,Hd2,Hd3]) + np.sqrt(2*lamda/beta)*np.random.normal(0,1)
        V_hat_value[i+1] = V_hat(theta_hat[:,i+1],omega1,omega2,omega3)
        #V_hat_value[i+1] = V_hat(theta_hat[:,i+1])
    
    res[s] = min(V_hat_value)
    min_theta_index = np.argmin(V_hat_value)
    mintheta[s] = theta_hat[0,min_theta_index]

task2_CVaR = min(res)
min_weight_index = np.argmin(res)
task2_VaR = mintheta[min_weight_index]
min_omega1,min_omega2,min_omega3 = allomega1[min_weight_index],allomega2[min_weight_index],allomega3[min_weight_index]


print('For asset1=N({},{}),asset2=N({},{}),asset3=N({},{})'.format(m1,s1,m2,s2,m3,s3))
print('VaR_SGLD is {:.4f}'.format(task2_VaR))
print('CVaR_SGLD is {:.4f}'.format(task2_CVaR))
print('weight for each asset: {:.4f},{:.4f},{:.4f}'.format(g(1,min_omega1,min_omega2,min_omega3),g(2,min_omega1,min_omega2,min_omega3),g(3,min_omega1,min_omega2,min_omega3)))

        

For asset1=N(0,1000.0),asset2=N(0,1),asset3=N(0,4)
VaR_SGLD is 0.7811
CVaR_SGLD is 46.4993
weight for each asset: 0.2430,0.4072,0.3498
